# Rotational Coordinate Scan

In the previous notebook we optimized the structure of butane. The lowest energy structure is the "anti" conformation. We know that as we turn the central C–C bond that we will pass through an energy barrier of an eclipsed conformation (with CH$_3$ eclipsed against hydrogen) and then settle into the energy well of the gauche conformation. If we keep turning the bond we will pass through the highest energy barrier of the conformation with both methyl groups eclipsed against each other to reach the other gauche conformation. Past that we would complete the full circle and reach the original anti conformation. 

Draw these conformers using Newman projection and convince yourself that the above is true.

## Bring Back Butane

First I will create the molecule object using the data from the optimized structure for butane. Consider the code below.

In [33]:
# use psi4conda environment
import psi4
import os
import numpy as np
import matplotlib.pyplot as plt

import helpers as hp # Many useful functions from https://lcbc-epfl.github.io/iesm-public/intro.html

output_file = "butane_PES_points_1.out"
psi4.set_memory("1GB")
psi4.set_output_file(output_file, append=False, loglevel=20, print_header=True, inherit_loglevel=True, execute=True)
psi4.core.set_num_threads(4)

# The Z-matrix as a text string ### From previous optimization
data = """
       0 1
       C       
       C             1    1.541090
       H             1    1.085997      2  110.737654
       H             1    1.086006      2  110.607207      3  119.996707
       H             1    1.085996      2  110.737537      3 -120.006541
       C             2    1.545066      1  112.446725      3   59.916254
       H             2    1.088519      1  109.298807      6  121.516910
       H             2    1.088518      1  109.301738      6 -121.519264
       C             6    1.541169      2  112.445221      1    dihedral
       H             6    1.088515      2  109.250918      9  121.551235
       H             6    1.088516      2  109.250874      9 -121.547269
       H             9    1.085998      6  110.604358     10   58.384210
       H             9    1.085992      6  110.745577     12  119.994957
       H             9    1.085994      6  110.746117     12 -119.995065
   
       dihedral  =  179.9414294137
       units angstrom
       """ 

# Create the Molecule object
mol = psi4.geometry(data)             # Create Molecule object from data string

energy_anti, hist = psi4.optimize("hf", molecule = mol, return_history=True)   # get energy of initial optimized structure
                                                                               # should I have used psi4.energy() instead?
print(f"Initial energy is {energy_anti:0.7f} Hartrees")

Optimizer: Optimization complete!
Initial energy is -155.4666462 Hartrees


## Finding Higher Ground

We can see if the structure will "fall" into the higher-energy gauche conformation and stay there if we start within that energy well. I will take the starting dihedral angle and twist it by 120 degrees (I will just subtract 120 from the current value). Then I will optimize the geometry so that the methyl groups can rotate to find the lowest-energy gauche conformation. Consider the code below.

In [34]:
psi4.set_options({
        "BASIS": "sto-3g",            # default => None - Basis set must be specified
#        "BASIS": "6-31+G(d)",            # default => None - Basis set must be specified
        "SAVE_OPTIMIZATION": True,    # default => False
        "OPT_TYPE": "min",            # default => "min":  "min","irc" or "ts"
        "MAXITER": 100,               # default => 50
        "GEOM_MAXITER": 100,          # default => 50
        "FULL_HESS_EVERY": -1,        # default => -1 -> only perform a calculation at the beginning
        "PRINT": 2,                   # default => 1
        "GUESS": "sad",               # default => auto
        "REFERENCE": "rhf",           # default => rhf
        "SCF_TYPE": "direct",         # default => pk
        "INTS_TOLERANCE": 1E-8,       # default => 1e-12. A value of 1e-8 is recommended when SCF_TYPE set to "direct"
        "PRINT_TRAJECTORY_XYZ_FILE":True,   # default => false
        "PRINT_OPT_PARAMS": True,     # default => False
        "WRITE_TRAJECTORY": True      # default => False
    })

mol2 = mol.clone()                    # make a copy of the molecule.

start_angle = mol2.get_variable("dihedral")
print(f"Before rotation: {start_angle:.3f} degrees") 

start_angle = start_angle - 120
print(f"After rotation: {start_angle:.3f} degrees\n") 
mol2.set_variable("dihedral", start_angle)


energy_gauche, hist = psi4.optimize("hf", molecule = mol2, return_history=True)
print(f"The optimized energy for this configuration is {opt_energy:.7f} Hartree\n")

mol2.print_out()
mol2.print_distances()
mol2.print_bond_angles()
mol2.print_in_input_format()

dihedral_angle = mol2.get_variable("dihedral")
#opt_length = mol.get_variable("length")

print(f"After optimization: {dihedral_angle:.3f} degrees\n") 

Before rotation: 179.941 degrees
After rotation: 59.941 degrees

Optimizer: Optimization complete!
The optimized energy for this configuration is -155.4653162 Hartree

After optimization: 66.508 degrees



### Comparing Anti and Gauche

Consider the code below. We will calculate the energy difference between the two conformations and also display the conformations side-by-side using another function in the ```helpers``` module. We see that the energy difference between anti and gauche is 3.5 kJ/mole. the literature value is between 3.6 and 3.8 kJ/mole. So our very simple calculation did a decent job of calculating the energy difference.

Examine the side-by-side comparison below. Note that the lowest energy geometry has a torsion angle of about 66.5 degrees rather than the cannonical value of 60 degrees expected for a gauche conformer. This is becasue of the steric bulk of the methyl groups. they push away from each other and the angle gets cheated slightly larger to lessen the amount of steric strain.

In [35]:
e_diff = b = ((energy_gauche - energy_anti) * psi4.constants.hartree2kJmol)
print(f"The energy difference is {e_diff:0.2f} kJ/mole")
hp.drawXYZSideBySide(mol, mol2)

The energy difference is 3.49 kJ/mole


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## Climbing the Summit

The path from anti to gauche and from gauche to the other gauche and then on back to anti desctibes a full circle for the torsion angle. We have found geometries that represent two stable conformations that are energy minima. We can convert from one to the other but must pass through a higher-energy structure to do so. How can we find this high-energy structure if the optimization algorithm want to find a way downhill at all times.

One way is to constrain the torsion angle. There are two possible eclipsed conformations. One is where methyl groups eclipse hydrogen groups and the other is when the methy groups eclipse each other. These will occur at torsion angles of 120 and zero degrees.

### A Molecular Eclipse

Consider the code below. We will set the torsion angle to 120 degrees. This will place both methyl groups in eclipse with a hydrogen group.



## Changing a Variable

One way to investigate the effect of changing bond angle on energy would be to take this optimized structure and simply assign new bond angles and then calculate the energy. This is a simplistic approach becuase one knows that the bond length will change slightly with bond angle due to changes in hydridization (e.g., $sp^2$ bonds are shorter than $sp^3$ bonds.) However let us use this simple way to start.

### Scanning the Bond Angle

We have set up the geometry with a variable named "angle." That variable has been changed in the optimization but we can change it manually using the ```.set_variable()``` method of the Molecule object. We will set the angle to whatever we want, then update the geomtry (recalculate the *xyz* cartesian coordinates with the new angle). After that we will calculate the energy of the new geometry. Rinse and repeat through a series of values.

Recall how we did this with *Gamess*. We took the inoput file that we had constructed using the optimized geometry and then used the awesome text tools of *Unix* to change the angle (*grep*, *sed* and *awk* are the secret underpinning of modern science.) Then we subitted a whole batch of input files with different bond angles into *Gamess* on-at-a-time using the batch-queueing program *GamessQ*. But with the power of *Python* we cane do all this inside this notebook. We are using *Psi4* because it has an *Python* API that can be imported and used in a notebook like this.

In the code below we pick values from a range of values and asign them in turn to the variable "angle." in each case we caluclate the energy and append that value to a list. After the series of calculations is complete we will have a list of angles and a list of energies.



In [27]:
mol.set_variable("dihedral", 60)
energy = psi4.energy("hf", molecule = mol)
print((energy - opt_energy) * psi4.constants.hartree2kJmol)
start_angle = mol.get_variable("angle")
print(f"Before optimization: {start_angle:.3f} degrees ") 

print(energy)
print(opt_energy)

hp.drawXYZ(mol)

5.806764110563818
Before optimization: 0.000 degrees 
-155.46443447863254
-155.46664615827854


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
output_file = "H2O_constrainst_2.out"
psi4.set_memory("1GB")
psi4.set_output_file(output_file, append=False, loglevel=20, print_header=True, inherit_loglevel=True, execute=True)
psi4.core.set_num_threads(4)

deviation = 10

start = opt_angle - deviation
end = opt_angle + deviation
number_of_points = 2 * deviation + 1
range_of_values = np.linspace(start, end, number_of_points) 

# We could also enter a set of values at which to to measure energies. try the range below and
# observe the energy profile. Can you explain it?
#range_of_values = [75,90,100,105,110,120,130,140,150,160,170,175,180,185,190,200,220,230,] 

angle_list = []
energy_list = []

for angle in range_of_values:
    mol.set_variable("angle", angle)
    energy = psi4.energy("hf", molecule = mol)
    angle_list.append(angle)
    energy_list.append(energy)

print(angle_list)
print(energy_list)

## Data Handling

We just generated some data. List of numbers are kind of useless. We need to do some data manipulation.

A convenient tool for data handling is the *Pandas* library. It provides a powerful data object, the dataframe. We will combine the two lists into a dataframe. Then we will print the dataframe. We can output the dataframe as a csv file for future analysis. In many ways, we can manipulate a dataframe like a spreadsheet. Consider the code below.

In [ ]:
import pandas as pd

data_dictionary = {"angle": angle_list,
                   "energy": energy_list}

df = pd.DataFrame.from_dict(data_dictionary)
#df.set_index('angle', inplace=True)
print(df.head())

### Math with a Dataframe

Lets do some math. We know the optimized energy from previous calulations above. We will subtract this lowets energy value from the energy column of the dataframe. Then we will multiply this column of relative energy values by a conversion factor to get kJ/mole rather than Hartrees.

In [ ]:
df["energy"] = df["energy"] - opt_energy
df["energy"] = df["energy"] * psi4.constants.hartree2kJmol

### A Quick Plot

The dataframe object is very powerful. It contains a method that will plot the data. In the example below, we set which columns were to be used in the x and y-axis of the plot. Does this look like a typical energy profile for a vibration? the angle decreasing and increasing on either side of the optimal value is just like a bending vibration.

In [ ]:
df.plot(x="angle", y = "energy", kind="line")   # try kind="scatter" as well

In [ ]:
dir(hp)

In [ ]:
import sys
sys.path.append("..")
sys.path

In [ ]:
import cclib

# The line in the output file that gives us problems has a series of "z" characters
# for the vesrion string. cclib chokes on this non-standard versioning
# We can run a grep command in terminal as shown below that will remove the
# line containing "zzzz" and write all the other lines into a new file.
# We will then use the edited cclib package (fixed another bug) to parse the output.
#!grep -v zzzz H2O_optimize.out > H2O_optimize2.out

output_file = "butane_1.out"

parser = cclib.io.ccopen(output_file)
data = parser.parse()

#print(f"There are {data.natom} atoms and {data.nmo} MOs")
#print("The vibrational frequencies are...")
#print(data.vibfreqs)
#print("The vibrational force constants are...")
#print(data.vibfconsts)
#print("The vibrational symmetry are...")
#print(data.vibsyms)


In [ ]:
dir(data)

In [ ]:
print("natom")
print(data.natom)
print("mult")
print(data.mult)
print("nbasis")
print(data.nbasis)
print("optdone")
print(data.optdone)
print("nelectrons")
print(data.nelectrons)
print("scfenergies")
print(data.scfenergies)
print("scfvalues")
#print(data.scfvalues)
print("new_geometries")
print(data.new_geometries)
print("geovalues")
print(data.geovalues)
print("OPT_DONE")
print(data.OPT_DONE)

print("unconverged_geometries")
print(data.unconverged_geometries)
print("unknown_geometries")
print(data.unknown_geometries)
print("converged_geometries")
print(data.converged_geometries)
print("atomcoords")
print(data.atomcoords)


In [ ]:
help(data)

In [ ]:
data.metadata

In [ ]:
psi4.core.variable('SCF TOTAL ENERGY')

In [ ]:
mol.to_dict()['elem']

In [ ]:
coordinates = hist["coordinates"]
traj = hp.mol2traj(mol, coordinates)
print(traj[22])

In [ ]:
hp.coord2traj_array(coordinates)

In [ ]:
hp.drawXYZGeomSlider(traj)

In [32]:
help(mol.clone)


Help on method clone in module psi4.core:

clone(...) method of psi4.core.Molecule instance
    clone(self: psi4.core.Molecule) -> psi4.core.Molecule
    
    Returns a new Molecule identical to arg1



In [21]:
dir(psi4.copy)

['Error',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_copy_dispatch',
 '_copy_immutable',
 '_deepcopy_atomic',
 '_deepcopy_dict',
 '_deepcopy_dispatch',
 '_deepcopy_list',
 '_deepcopy_method',
 '_deepcopy_tuple',
 '_keep_alive',
 '_reconstruct',
 'copy',
 'deepcopy',
 'dispatch_table',
 'error']